In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-louvain
!pip install karateclub

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 1.8MB 7.7MB/s 
     |████████████████████████████████| 24.2MB 44.8MB/s 
  Created wheel for karateclub: filename=karateclub-1.0.22-cp36-none-any.whl size=93242 sha256=04bf37fb703f6512baaedf75d5dea54d003a11ebc7a9d0892b3949aac6c125f2
  Stored in directory: /root/.cache/pip/wheels/60/ef/0a/b9f163e1bc2fa9337ef4b183529d09c2ea78588cbc2c3753ef
Successfully built karateclub
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from karateclub.community_detection.non_overlapping import LabelPropagation
import itertools
from networkx.algorithms.community.label_propagation import label_propagation_communities as LP
import time
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/RS_Doc/data  time user/timeuser_train_7.csv')
df = df.drop(columns=['Unnamed: 0'])

df_test = pd.read_csv('/content/drive/MyDrive/RS_Doc/data  time user/timeuser_test_3.csv')
df_test = df_test.drop(columns=['Unnamed: 0', 'dt'])

In [ ]:
weighted_edges = df.drop(columns=['dt'])
weights = np.ones((len(df)))
weighted_edges['w'] = weights
weighted_edges.head()

,userId,followId
0,9b670e4c393b771d904c3eaafa05e55f,c0fd9e1a93f3db44f5f9499552122f91
1,9b670e4c393b771d904c3eaafa05e55f,0354af7e148421884f15210915802a90
2,9b670e4c393b771d904c3eaafa05e55f,b1840b957592d5e514960b140c54af46
3,9b670e4c393b771d904c3eaafa05e55f,7f27338f8cb2bf2bfd6aafd7729b9bef
4,9b670e4c393b771d904c3eaafa05e55f,3db47a38e543d48e0cfa8b2c81aee13f


In [ ]:
user_follow_train = {}
for i in range(len(df)):
    userId = df['userId'].iloc[i]
    followId = df['followId'].iloc[i]
    
    if userId not in user_follow_train:
        user_follow_train[userId] = []
    if followId not in user_follow_train:
        user_follow_train[followId] = []

    if followId not in user_follow_train[userId]: 
        user_follow_train[userId].append(followId)
    if userId not in user_follow_train[followId]:
        user_follow_train[followId].append(userId)


user_follow_test = {}
for i in range(len(df_test)):
    userId = df_test['userId'].iloc[i]
    followId = df_test['followId'].iloc[i]
    
    if userId not in user_follow_test:
        user_follow_test[userId] = []
    if followId not in user_follow_test:
        user_follow_test[followId] = []

    if followId not in user_follow_test[userId]: 
        user_follow_test[userId].append(followId)
    if userId not in user_follow_test[followId]:
        user_follow_test[followId].append(userId)

In [ ]:
G = nx.Graph()
G.add_nodes_from(df['userId'].unique())
G.add_weighted_edges_from(weighted_edges.values.tolist())

## Louvain

In [ ]:
# compute the best partition
partition = community_louvain.best_partition(G)

In [ ]:
key_list = list(partition.keys())
val_list = list(partition.values())
key_list = np.asarray(key_list)
val_list = np.asarray(val_list)

In [ ]:
recommend_list = {}
begin = time.time()
for id in range(np.max(np.asarray(list(partition.values())))+1):
    nodes = key_list[val_list == id]
    if id % 100 == 0:
        print(f"Making recommendation for person {id}")
    for node in nodes:
        node_recommend = nodes[nodes != node]
        recommend_list[node] = node_recommend
end = time.time()
print((end-begin)/60)

Making recommendation for person 0
Making recommendation for person 100
Making recommendation for person 200
Making recommendation for person 300
Making recommendation for person 400
Making recommendation for person 500
Making recommendation for person 600
Making recommendation for person 700
Making recommendation for person 800
Making recommendation for person 900
0.037541091442108154


In [ ]:
top_user = {}

for user in tqdm(key_list):
    top_user[user] = 0

for id in tqdm(range(len(df))):
    top_user[df['followId'].iloc[id]] += 1

top = min(len(list(dict(sorted(top_user.items(), key=lambda item: item[1])).keys())), 5)
top_follow = list(dict(sorted(top_user.items(), key=lambda item: item[1])).keys())[-top:]

100%|██████████| 36540/36540 [00:00<00:00, 84500.06it/s]


In [ ]:
# mAP = 0
# mAR = 0
# mF1 = 0
# num_per = 0

# for user in df_test['userId'].unique():
#     # user khong co trong tap train
#     if user not in recommend_list.keys():
#         continue

#     recommends = recommend_list[user]
#     if len(recommends) < 5:
#         recommends = list(recommends)
#         recommends.extend(top_follow)
#         recommends = np.asarray(recommends)

#     follows = user_follow_test[user]
#     TP = len(set(recommends)&set(follows))
#     TPP = len(recommends) - len(set(recommends)&set(user_follow_train[user]))
#     TAP = len(follows)

#     # user da follow het cmn community
#     if TPP == 0:
#         continue

#     num_per += 1
#     precision = TP/TPP
#     recall = TP/TAP
#     mAP += precision
#     mAR += recall
#     mF1 += 2*precision*recall/(precision+recall+0.0001)
#     # print(precision, recall)

In [ ]:
mAP = 0
mAR = 0
mF1 = 0
num_per = 0

for user in df_test['userId'].unique():
    # user khong co trong tap train
    if user not in recommend_list.keys():
        continue

    recommends = recommend_list[user]
    follows = user_follow_test[user]
    TP = len(set(recommends)&set(follows))
    TPP = len(recommends) - len(set(recommends)&set(user_follow_train[user]))
    TAP = len(follows)

    # user da follow het cmn community
    if TPP == 0:
        continue

    num_per += 1
    precision = TP/TPP
    recall = TP/TAP
    mAP += precision
    mAR += recall
    mF1 += 2*precision*recall/(precision+recall+0.0001)
    # print(precision, recall)

In [ ]:
print(f"mAP = {mAP/num_per}, mAR = {mAR/num_per}, mF1 = {mF1/num_per}")

mAP = 0.006553920641586211, mAR = 0.39300281937225146, mF1 = 0.009002545784910993


## Label Propagation

In [ ]:
part_gen = LP(G)

communities = []
for community in part_gen:
    communities.append(list(community))

In [ ]:
recommend_list = {}

begin = time.time()
for community in communities:
    community = np.asarray(community)
    for node in community:
        recommend_nodes = community[community != node]
        recommend_list[node] = recommend_nodes
end = time.time()
print((end-begin)/60)

0.1261385639508565


In [ ]:
mAP = 0
mAR = 0
mF1 = 0
num_per = 0

for user in df_test['userId'].unique():
    # user khong co trong tap train
    if user not in recommend_list.keys():
        continue

    recommends = recommend_list[user]
    follows = user_follow_test[user]
    TP = len(set(recommends)&set(follows))
    TPP = len(recommends) - len(set(recommends)&set(user_follow_train[user]))
    TAP = len(follows)

    # user da follow het cmn community
    if TPP == 0:
        continue

    num_per += 1
    precision = TP/TPP
    recall = TP/TAP
    mAP += precision
    mAR += recall
    mF1 += 2*precision*recall/(precision+recall+0.0001)
    # print(precision, recall)

In [ ]:
top_user = {}

for user in tqdm(key_list):
    top_user[user] = 0

for id in tqdm(range(len(df))):
    top_user[df['followId'].iloc[id]] += 1

top = min(len(list(dict(sorted(top_user.items(), key=lambda item: item[1])).keys())), 2)
top_follow = list(dict(sorted(top_user.items(), key=lambda item: item[1])).keys())[-top:]

100%|██████████| 36540/36540 [00:00<00:00, 84194.84it/s]


In [ ]:
# mAP = 0
# mAR = 0
# mF1 = 0
# num_per = 0

# for user in df_test['userId'].unique():
#     # user khong co trong tap train
#     if user not in recommend_list.keys():
#         continue

#     recommends = recommend_list[user]
#     if len(recommends) < 5:
#         recommends = list(recommends)
#         recommends.extend(top_follow)
#         recommends = np.asarray(recommends)

#     follows = user_follow_test[user]
#     TP = len(set(recommends)&set(follows))
#     TPP = len(recommends) - len(set(recommends)&set(user_follow_train[user]))
#     TAP = len(follows)

#     # user da follow het cmn community
#     if TPP == 0:
#         continue

#     num_per += 1
#     precision = TP/TPP
#     recall = TP/TAP
#     mAP += precision
#     mAR += recall
#     mF1 += 2*precision*recall/(precision+recall+0.0001)

In [ ]:
print(f"mAP = {mAP/num_per}, mAR = {mAR/num_per}, mF1 = {mF1/num_per}")

mAP = 0.06511470224989487, mAR = 0.3312133133837014, mF1 = 0.027127367034854397
